In [1]:
import numpy as np
import pandas as pd
import torch
import tensorly as tl
# library for TTD   (Tensorly  (search))

from tensorly.decomposition import parafac
from tensorly.decomposition import tucker
from tensorly.decomposition import tensor_train
from tensorly.decomposition import tensor_train
# different forms of tensor
from sklearn.preprocessing import MinMaxScaler
from tensorly import tt_to_tensor
from tensorly.decomposition import matrix_product_state
# tensor operations


#from math import ceil
#from google.colab import drive
#drive.mount('/content/gdrive')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
import time
import os
import copy
import torch.nn.functional as Fun
print("PyTorch Version:", torch.__version__)
print("Torchvision Version:", torchvision.__version__)
print("GPU is available?", torch.cuda.is_available())

PyTorch Version: 1.13.1
Torchvision Version: 0.14.1
GPU is available? True


In [2]:
dtype = torch.float64
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
features = list()
with open("/home/c/cl237/TenBCD/UCI HAR/data/features.txt") as f:
    features = [line.split()[1] for line in f.readlines()]
print('# of Features: {}'.format(len(features)))

# of Features: 561


## get train data

In [4]:
pd_X_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_train.txt', delim_whitespace=True, header=None)
pd_X_train.columns = features
pd_y_train = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_train.txt', delim_whitespace=True, names=['Activity'])

N = len(pd_X_train)

K = 6

x = pd_X_train.values
y = pd_y_train.values

X_train=torch.tensor(x,dtype = torch.float64,device=device)
y_train=torch.tensor(y,dtype = int) #dtype=torch.int,device=device)

y_train = torch.flatten(y_train)

X_train = torch.t(X_train)

y_one_hot = torch.zeros(N, K).scatter_(1, torch.reshape(y_train-1, (N, 1)), 1)
y_one_hot = torch.t(y_one_hot).to(device=device)
y_train = y_train.to(device=device)

## get test data

In [5]:
pd_X_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/X_test.txt', delim_whitespace=True, header=None)
pd_X_test.columns = features
pd_y_test = pd.read_csv('/home/c/cl237/TenBCD/UCI HAR/data/y_test.txt', delim_whitespace=True, names=['Activity'])

N_test = len(pd_X_test)

x = pd_X_test.values
y = pd_y_test.values

X_test=torch.tensor(x,dtype = torch.float64, device=device)
y_test=torch.tensor(y,dtype = int)

y_test = torch.flatten(y_test)

X_test = torch.t(X_test)

y_test_one_hot = torch.zeros(N_test, K).scatter_(1, torch.reshape(y_test-1, (N_test, 1)), 1)
y_test_one_hot = torch.t(y_test_one_hot).to(device=device)
y_test = y_test.to(device=device)

# Main Algorithm

In [6]:
def updateV(U1,U2,W,b,rho,gamma): 
    _, d = W.size()
    I = torch.eye(d, device=device) #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
    U1 = nn.ReLU()(U1)
    _, col_U2 = U2.size()
    Vstar = torch.mm(torch.inverse(rho*(torch.mm(torch.t(W),W))+gamma*I), rho*torch.mm(torch.t(W),U2-b.repeat(1,col_U2))+gamma*U1)
    # check how to get V* W*
    # torch.mm*=(input,mat2) performs a matrix multiplication of the matrices input and mat2.
    #   if input is a (n*m) tensor,mat2 is a (m*p) tensor, output will be (n*p) tensor
    return Vstar

In [7]:
def updateWb_org(U, V, W, b, alpha, rho): 
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse(alpha*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

In [8]:
def updateWb(U, V, W, b, W_tensor_rec, alpha, rho,tau): 
    W_tensor_rec = torch.as_tensor(W_tensor_rec,device=device).float()
    W_tensor2matrix = W_tensor_rec.reshape(W.shape)
    d,N = V.size()
    I = torch.eye(d, device=device)
    _, col_U = U.size()
    Wstar = torch.mm(alpha*W+tau*W_tensor2matrix+rho*torch.mm(U-b.repeat(1,col_U),torch.t(V)),torch.inverse((alpha+tau)*I+rho*(torch.mm(V,torch.t(V)))))
    bstar = 0*(alpha*b+rho*torch.sum(U-torch.mm(Wstar,V), dim=1).reshape(b.size()))/(rho*N+alpha)
    return Wstar, bstar

In [9]:
# U update  E(1) in global convergent
def relu_prox(a, b, gamma, d, N):
    val = torch.empty(d,N, device=device)
    x = (a+gamma*b)/(1+gamma)
    y = torch.min(b,torch.zeros(d,N, device=device))

    val = torch.where(a+gamma*b < 0, y, torch.zeros(d,N, device=device))
    val = torch.where(((a+gamma*b >= 0) & (b >=0)) | ((a*(gamma-np.sqrt(gamma*(gamma+1))) <= gamma*b) & (b < 0)), x, val)
    val = torch.where((-a <= gamma*b) & (gamma*b <= a*(gamma-np.sqrt(gamma*(gamma+1)))), b, val)
    return val

# Model Initialization and training/testing

In [ ]:

niter = 500
rank = 80
tau = 3
alpha = 1



for gamma in (0.01,0.1,0.5,1,5,10,100): #(0.01,0.1,0.5,1,5,10,100)
    for rho in (0.01,0.1,0.5,1,5,10,100): #(0.01,0.1,0.5,1,5,10,100)

        print ("rank=",rank, "tau=",tau, "gamma=",gamma, "rho=",rho, "alpha",alpha)

        loss1 = np.empty(niter)
        loss2 = np.empty(niter)
        accuracy_train = np.empty(niter)
        accuracy_test = np.empty(niter)
        time1 = np.empty(niter)

        results = torch.zeros(1, 5, niter)



        for Out_iter in range(1):
            rank_initial = 400
            seed = 10 + 10*Out_iter
            torch.manual_seed(seed)
            if torch.cuda.is_available():
                torch.manual_seed(seed)
                
            d0 = 561 #561 =3*11*17
            d1 =  1024    # 2^10
            d2 =  1024  
            d3 = 1024
            d4 = 512      # 2^9
            d5 = 512
            d6 = 6 


            W1 = 0.01*torch.randn(d1, d0,dtype = torch.float64, device=device)
            W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
            W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
            factors1 = tensor_train(W1_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
            W1_tl_tensor_rec = tt_to_tensor(factors1)
            b1 = 0*torch.ones(d1, 1,dtype = torch.float64, device=device) # 0 is stable


            W2 = 0.01*torch.randn(d2, d1,dtype = torch.float64, device=device)
            W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
            W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy())
            factors2 = tensor_train(W2_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
            W2_tl_tensor_rec = tt_to_tensor(factors2)     
            b2 = 0*torch.ones(d2, 1,dtype = torch.float64, device=device)


            W3 = 0.01*torch.randn(d3, d2,dtype = torch.float64, device=device)
            W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
            W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())
            factors3 = tensor_train(W3_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
            W3_tl_tensor_rec = tt_to_tensor(factors3)
            b3 = 0*torch.ones(d3, 1,dtype = torch.float64, device=device)

            W4 = 0.01*torch.randn(d4, d3, dtype = torch.float64,device=device)
            W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2)) # 8 number of 4s, 2 number of 8s
            W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())
            factors4 = tensor_train(W4_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
            W4_tl_tensor_rec = tt_to_tensor(factors4)
            b4 = 0*torch.ones(d4, 1, dtype = torch.float64,device=device)

            W5 = 0.01*torch.randn(d5, d4, dtype = torch.float64,device=device)
            W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4)) # 8 number of 4s, 2 number of 8s
            W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())
            factors5 = tensor_train(W5_tl_tensor, (1, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial, rank_initial,1))
            W5_tl_tensor_rec = tt_to_tensor(factors5)
            b5 = 0*torch.ones(d5, 1, dtype = torch.float64,device=device)


            W6 = 0.01*torch.randn(d6, d5, dtype = torch.float64,device=device)
            b6 = 0*torch.ones(d6, 1, dtype = torch.float64,device=device)



            U1 = torch.addmm(b1.repeat(1, N), W1, X_train)
            V1 = nn.ReLU()(U1)
            U2 = torch.addmm(b2.repeat(1, N), W2, V1)
            V2 = nn.ReLU()(U2)
            U3 = torch.addmm(b3.repeat(1, N), W3, V2)
            V3 = nn.ReLU()(U3)
            U4 = torch.addmm(b4.repeat(1, N), W4, V3)
            V4 = nn.ReLU()(U4)
            U5 = torch.addmm(b5.repeat(1, N), W5, V4)
            V5 = nn.ReLU()(U5)
            U6 = torch.addmm(b6.repeat(1, N), W6, V5)
            V6 = U6 
            # U4 = torch.addmm(b4.repeat(1, N), W4, V3)
            # V4 = U4



            # Iterations
            print('Train on', N, 'samples, validate on', N_test, 'samples')
            for k in range(niter):
                start = time.time()

         # update for last layer
                # update V6
                V6 = (y_one_hot + gamma*U6 + alpha*V6)/(1 + gamma + alpha)

                # update U6 
                U6 = (gamma*V6 + rho*(torch.mm(W6,V5) + b6.repeat(1,N)))/(gamma + rho)

                # update W6 and b6
                W6, b6 = updateWb_org(U6,V5,W6,b6,alpha,rho)



         # update for 5th layer
                # update V5
                V5 = updateV(U5,U6,W6,b6,rho,gamma)

                # update U5
                U5 = relu_prox(V5,(rho*torch.addmm(b5.repeat(1,N), W5, V4) + alpha*U5)/(rho + alpha),(rho + alpha)/gamma,d5,N)

                # update W5 and b5
                W5, b5 = updateWb(U5,V4,W5,b5,W5_tl_tensor_rec, alpha,rho,tau)

                # G update (TTD)   
                W5_torch_tensor = W5.reshape((4,4,4,4,4,4,4,4,4))
                W5_tl_tensor = tl.tensor(W5_torch_tensor.cpu().numpy())  # transfer tensorly package
                factors5 = tensor_train(W5_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,1))
                #set of tensor cores
                W5_tl_tensor_rec = tt_to_tensor(factors5)


          # update for 4th layer
                # update V4
                V4 = updateV(U4,U5,W5,b5,rho,gamma)

                # update U4
                U4 = relu_prox(V4,(rho*torch.addmm(b4.repeat(1,N), W4, V3) + alpha*U4)/(rho + alpha),(rho + alpha)/gamma,d4,N)

                # update W4 and b4
                W4, b4 = updateWb(U4,V3,W4,b4,W4_tl_tensor_rec, alpha,rho,tau)

                # G update (TTD)   
                W4_torch_tensor = W4.reshape((4,4,4,4,4,4,4,4,4,2))
                W4_tl_tensor = tl.tensor(W4_torch_tensor.cpu().numpy())  # transfer tensorly package
                factors4 = tensor_train(W4_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
                #set of tensor cores
                W4_tl_tensor_rec = tt_to_tensor(factors4)

          # update for 3nd layer
                # update V3
                V3 = updateV(U3,U4,W4,b4,rho,gamma)

                # update U3
                U3 = relu_prox(V3,(rho*torch.addmm(b3.repeat(1,N), W3, V2) + alpha*U3)/(rho + alpha),(rho + alpha)/gamma,d3,N)

                # update W3 and b3
                W3, b3 = updateWb(U3,V2,W3,b3,W3_tl_tensor_rec, alpha,rho,tau)

                # G update (TTD)   
                W3_torch_tensor = W3.reshape((4,4,4,4,4,4,4,4,4,4))
                W3_tl_tensor = tl.tensor(W3_torch_tensor.cpu().numpy())  # transfer tensorly package
                factors3 = tensor_train(W3_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
                #set of tensor cores
                W3_tl_tensor_rec = tt_to_tensor(factors3)

         # update for 2nd layer
                # update V2
                V2 = updateV(U2,U3,W3,b3,rho,gamma)

                # update U2
                U2 = relu_prox(V2,(rho*torch.addmm(b2.repeat(1,N), W2, V1) + alpha*U2)/(rho + alpha),(rho + alpha)/gamma,d2,N)

                # update W2 and b2
                W2, b2 = updateWb(U2,V1,W2,b2,W2_tl_tensor_rec, alpha,rho,tau)

                # G update (TTD)   
                W2_torch_tensor = W2.reshape((4,4,4,4,4,4,4,4,4,4))
                W2_tl_tensor = tl.tensor(W2_torch_tensor.cpu().numpy()) 
                factors2 = tensor_train(W2_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
                W2_tl_tensor_rec = tt_to_tensor(factors2)


        # update for 1st layer
                # update V1
                V1 = updateV(U1,U2,W2,b2,rho,gamma)

                # update U1
                U1 = relu_prox(V1,(rho*torch.addmm(b1.repeat(1,N), W1, X_train) + alpha*U1)/(rho + alpha),(rho + alpha)/gamma,d1,N)

                # update W1 and b1
                W1, b1 = updateWb(U1,X_train,W1,b1,W1_tl_tensor_rec, alpha,rho,tau)

                # G update
                W1_torch_tensor = W1.reshape((6,22,34,2,2,2,2,2,2,2))
                W1_tl_tensor = tl.tensor(W1_torch_tensor.cpu().numpy())
                factors1 = tensor_train(W1_tl_tensor, (1,rank,rank,rank,rank,rank,rank,rank,rank,rank,1))
                W1_tl_tensor_rec = tt_to_tensor(factors1)




         # prediction for trainning data
                a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).double(), X_train))
                #a1_train = nn.ReLU()(torch.addmm(b1.repeat(1, N), W1, x_train))
                a2_train = nn.ReLU()(torch.addmm(b2.repeat(1, N), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).double(), a1_train))
                a3_train = nn.ReLU()(torch.addmm(b3.repeat(1, N), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).double(), a2_train))
                a4_train = nn.ReLU()(torch.addmm(b4.repeat(1, N), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).double(), a3_train))
                a5_train = nn.ReLU()(torch.addmm(b5.repeat(1, N), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).double(), a4_train))
                pred = torch.argmax(torch.addmm(b6.repeat(1, N), W6, a5_train), dim=0)
                # check argmax and addmm, dim=0

         #Prediction for test data
                a1_test = nn.ReLU()(torch.addmm(b1.repeat(1, N_test), torch.as_tensor(W1_tl_tensor_rec,device=device).reshape((d1, d0)).double(), X_test))
                a2_test = nn.ReLU()(torch.addmm(b2.repeat(1, N_test), torch.as_tensor(W2_tl_tensor_rec,device=device).reshape((d2, d1)).double(), a1_test))
                a3_test = nn.ReLU()(torch.addmm(b3.repeat(1, N_test), torch.as_tensor(W3_tl_tensor_rec,device=device).reshape((d3, d2)).double(), a2_test))
                a4_test = nn.ReLU()(torch.addmm(b4.repeat(1, N_test), torch.as_tensor(W4_tl_tensor_rec,device=device).reshape((d4, d3)).double(), a3_test))
                a5_test = nn.ReLU()(torch.addmm(b5.repeat(1, N_test), torch.as_tensor(W5_tl_tensor_rec,device=device).reshape((d5, d4)).double(), a4_test))
                pred_test = torch.argmax(torch.addmm(b6.repeat(1, N_test), W6, a5_test), dim=0)






           #emperical loss
                loss1[k] = gamma/2*torch.pow(torch.dist(V6,y_one_hot,2),2).cpu().numpy()
                # torch.pow:Takes the power of each element in input with exponent and returns a tensor with the result.
                # torch.disk: Returns the p-norm of (input - other)
                # Eq (5) in paper
                loss2[k] = (
                    loss1[k] 
                    +rho/2*torch.pow(torch.dist(torch.addmm(b1.repeat(1,N), W1, X_train),U1,2),2).cpu().numpy() 
                    +rho/2*torch.pow(torch.dist(torch.addmm(b2.repeat(1,N), W2, V1),U2,2),2).cpu().numpy() 
                    +rho/2*torch.pow(torch.dist(torch.addmm(b3.repeat(1,N), W3, V2),U3,2),2).cpu().numpy() 
                    +rho/2*torch.pow(torch.dist(torch.addmm(b4.repeat(1,N), W4, V3),U4,2),2).cpu().numpy() 
                    +rho/2*torch.pow(torch.dist(torch.addmm(b5.repeat(1,N), W5, V4),U5,2),2).cpu().numpy() 
                    +rho/2*torch.pow(torch.dist(torch.addmm(b6.repeat(1,N), W6, V5),U6,2),2).cpu().numpy() 
                    + gamma/2*torch.pow(torch.dist(V1,nn.ReLU()(U1),2),2).cpu().numpy() 
                    + gamma/2*torch.pow(torch.dist(V2,nn.ReLU()(U2),2),2).cpu().numpy() 
                    + gamma/2*torch.pow(torch.dist(V3,nn.ReLU()(U3),2),2).cpu().numpy() 
                    + gamma/2*torch.pow(torch.dist(V4,nn.ReLU()(U4),2),2).cpu().numpy() 
                    + gamma/2*torch.pow(torch.dist(V5,nn.ReLU()(U5),2),2).cpu().numpy() 
                    + gamma/2*torch.pow(torch.dist(V6,U6,2),2).cpu().numpy() 
                    +tau/2*torch.pow(torch.dist(W1.reshape((6,22,34,2,2,2,2,2,2,2)),torch.as_tensor(W1_tl_tensor_rec,device=device).double(),2),2).cpu().numpy() 
                    +tau/2*torch.pow(torch.dist(W2.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W2_tl_tensor_rec,device=device).double(),2),2).cpu().numpy() 
                    +tau/2*torch.pow(torch.dist(W3.reshape((4,4,4,4,4,4,4,4,4,4)),torch.as_tensor(W3_tl_tensor_rec,device=device).double(),2),2).cpu().numpy() 
                    +tau/2*torch.pow(torch.dist(W4.reshape((4,4,4,4,4,4,4,4,4,2)),torch.as_tensor(W4_tl_tensor_rec,device=device).double(),2),2).cpu().numpy() 
                    +tau/2*torch.pow(torch.dist(W5.reshape((4,4,4,4,4,4,4,4,4)),torch.as_tensor(W5_tl_tensor_rec,device=device).double(),2),2).cpu().numpy()
                           )
 

                # compute training accuracy
                correct_train = pred == y_train-1
                accuracy_train[k] = np.mean(correct_train.cpu().numpy())

                # compute validation accuracy
                correct_test = pred_test == y_test-1
                accuracy_test[k] = np.mean(correct_test.cpu().numpy())

                # compute training time
                stop = time.time()
                duration = stop - start
                time1[k] = duration

                # print results
                print('Repeatition', Out_iter + 1, 'Epoch', k + 1, '/', niter, '\n', 
                      '-', 'time:', time1[k], '-', 'sq_loss:', loss1[k], '-', 'tot_loss:', loss2[k], 
                      '-', 'acc:', accuracy_train[k], '-', 'val_acc:', accuracy_test[k])

        ##############
        ############## compute CR
            factors1_shape=[f.shape for f in factors1]
            Sum_of_variables_factors1=sum(list(x*y*z for x,y,z in factors1_shape))
            factors2_shape=[f.shape for f in factors2]
            Sum_of_variables_factors2=sum(list(x*y*z for x,y,z in factors2_shape))
            factors3_shape=[f.shape for f in factors3]
            Sum_of_variables_factors3=sum(list(x*y*z for x,y,z in factors3_shape))
            factors4_shape=[f.shape for f in factors4]
            Sum_of_variables_factors4=sum(list(x*y*z for x,y,z in factors4_shape))
            factors5_shape=[f.shape for f in factors5]
            Sum_of_variables_factors5=sum(list(x*y*z for x,y,z in factors5_shape))

            total_variabels=Sum_of_variables_factors1+Sum_of_variables_factors2+Sum_of_variables_factors3+Sum_of_variables_factors4+Sum_of_variables_factors5

            CR_1=((total_variabels)+(d5*d6))/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5+d5*d6)
            CR_2=(total_variabels)/(d0*d1+d1*d2+d2*d3+d3*d4+d4*d5)
            print("CR_1 = "+str(CR_1)+"   "+"CR_2 = " +str(CR_2))


            results[Out_iter,0,:] = torch.tensor(loss1)
            results[Out_iter,1,:] = torch.tensor(loss2)
            results[Out_iter,2,:] = torch.tensor(accuracy_train)
            results[Out_iter,3,:] = torch.tensor(accuracy_test)
            results[Out_iter,4,:] = torch.tensor(time1)
            CR=(CR_1,CR_2)



rank= 80 tau= 3 gamma= 0.01 rho= 0.01 alpha 1
Train on 7352 samples, validate on 2947 samples
Repeatition 1 Epoch 1 / 500 
 - time: 4.632505178451538 - sq_loss: 9.281772548731674 - tot_loss: 364.645444451347 - acc: 0.20919477693144722 - val_acc: 0.1998642687478792
Repeatition 1 Epoch 2 / 500 
 - time: 1.7377803325653076 - sq_loss: 2.3666199783600685 - tot_loss: 41.757585129313284 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 3 / 500 
 - time: 1.6907775402069092 - sq_loss: 0.6147995052644343 - tot_loss: 19.91272075101563 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 4 / 500 
 - time: 1.7081146240234375 - sq_loss: 0.1649341814275742 - tot_loss: 15.280431608960264 - acc: 0.191512513601741 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 5 / 500 
 - time: 1.6893768310546875 - sq_loss: 0.04601343080860084 - tot_loss: 12.445708886624802 - acc: 0.2089227421109902 - val_acc: 0.18934509670851712
Repeatition 1 Epoch 6 / 500 
 - time

Repeatition 1 Epoch 47 / 500 
 - time: 1.7955260276794434 - sq_loss: 2.3555301647859164e-06 - tot_loss: 2.4691847374718994 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 48 / 500 
 - time: 1.8137233257293701 - sq_loss: 2.277348483658287e-06 - tot_loss: 2.4222280374395697 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 49 / 500 
 - time: 1.9416093826293945 - sq_loss: 2.2033596997050126e-06 - tot_loss: 2.3766256663933203 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 50 / 500 
 - time: 1.7880349159240723 - sq_loss: 2.133251602461609e-06 - tot_loss: 2.33231560576336 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 51 / 500 
 - time: 1.9617369174957275 - sq_loss: 2.0667412619118226e-06 - tot_loss: 2.289239617277821 - acc: 0.1749183895538629 - val_acc: 0.166610111978283
Repeatition 1 Epoch 52 / 500 
 - time: 2.0515334606170654 - sq_loss: 2.003571903844086e-06 - tot_loss: 2.2473434166

Repeatition 1 Epoch 94 / 500 
 - time: 1.6961641311645508 - sq_loss: 7.722007408466604e-07 - tot_loss: 1.146636535340934 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 95 / 500 
 - time: 1.693389892578125 - sq_loss: 7.588625209945393e-07 - tot_loss: 1.1304916704237074 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 96 / 500 
 - time: 1.7315363883972168 - sq_loss: 7.458779365085494e-07 - tot_loss: 1.1146555448621238 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 97 / 500 
 - time: 1.7369420528411865 - sq_loss: 7.332345822220646e-07 - tot_loss: 1.099121027266753 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 98 / 500 
 - time: 1.7215824127197266 - sq_loss: 7.209205538766392e-07 - tot_loss: 1.083881146205481 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 99 / 500 
 - time: 1.7094659805297852 - sq_loss: 7.089244021414184e-07 - tot_loss: 1.0689

Repeatition 1 Epoch 140 / 500 
 - time: 1.731510877609253 - sq_loss: 3.9722544893771806e-07 - tot_loss: 0.6406693665251723 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 141 / 500 
 - time: 1.7200777530670166 - sq_loss: 3.9251129280256524e-07 - tot_loss: 0.6335590505836083 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 142 / 500 
 - time: 1.7483482360839844 - sq_loss: 3.878867590717933e-07 - tot_loss: 0.6265651081403465 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 143 / 500 
 - time: 1.7062456607818604 - sq_loss: 3.833496056731172e-07 - tot_loss: 0.6196853897354677 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 144 / 500 
 - time: 1.7180540561676025 - sq_loss: 3.788977094619424e-07 - tot_loss: 0.6129177461593319 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 145 / 500 
 - time: 1.7263853549957275 - sq_loss: 3.7452903977933973e-07 - tot_

Repeatition 1 Epoch 186 / 500 
 - time: 1.7109696865081787 - sq_loss: 2.478280288786622e-07 - tot_loss: 0.40633345151348227 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 187 / 500 
 - time: 1.6896939277648926 - sq_loss: 2.456588065515682e-07 - tot_loss: 0.40281733023519467 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 188 / 500 
 - time: 1.738356590270996 - sq_loss: 2.435209651713872e-07 - tot_loss: 0.39935009267283617 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 189 / 500 
 - time: 1.7208850383758545 - sq_loss: 2.414138542087154e-07 - tot_loss: 0.39593085634338904 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 190 / 500 
 - time: 1.710212230682373 - sq_loss: 2.393368395784729e-07 - tot_loss: 0.39255874510069894 - acc: 0.1913764961915125 - val_acc: 0.18221920597217509
Repeatition 1 Epoch 191 / 500 
 - time: 1.7326889038085938 - sq_loss: 2.3728930188290933e-07 - to

...

Repeatition 1 Epoch 215 / 500 
 - time: 1.8241360187530518 - sq_loss: 0.0005743602484762567 - tot_loss: 1.51109307491458 - acc: 0.5941240478781284 - val_acc: 0.5917882592466915
Repeatition 1 Epoch 216 / 500 
 - time: 1.8134238719940186 - sq_loss: 0.0005708880399603505 - tot_loss: 1.5002727342980344 - acc: 0.5957562568008705 - val_acc: 0.5924669155072956
Repeatition 1 Epoch 217 / 500 
 - time: 1.8555800914764404 - sq_loss: 0.0005674873305474722 - tot_loss: 1.4895571830306196 - acc: 0.5971164309031556 - val_acc: 0.5934848998982015
Repeatition 1 Epoch 218 / 500 
 - time: 1.8539128303527832 - sq_loss: 0.000564155570655489 - tot_loss: 1.4789450669354645 - acc: 0.5979325353645266 - val_acc: 0.5941635561588056
Repeatition 1 Epoch 219 / 500 
 - time: 1.8361256122589111 - sq_loss: 0.0005608902918187443 - tot_loss: 1.4684348447045623 - acc: 0.5988846572361263 - val_acc: 0.5955208686800135
Repeatition 1 Epoch 220 / 500 
 - time: 1.8469691276550293 - sq_loss: 0.0005576893763098933 - tot_loss: 1.45

Repeatition 1 Epoch 261 / 500 
 - time: 1.8243060111999512 - sq_loss: 0.0004642202553897986 - tot_loss: 1.1042901892658363 - acc: 0.6429542981501633 - val_acc: 0.6399728537495758
Repeatition 1 Epoch 262 / 500 
 - time: 1.8260283470153809 - sq_loss: 0.00046257139906483614 - tot_loss: 1.0971632361546222 - acc: 0.6440424374319913 - val_acc: 0.6403121818798778
Repeatition 1 Epoch 263 / 500 
 - time: 1.7891652584075928 - sq_loss: 0.00046094266621545443 - tot_loss: 1.0900964684365424 - acc: 0.6447225244831338 - val_acc: 0.6413301662707839
Repeatition 1 Epoch 264 / 500 
 - time: 1.8272309303283691 - sq_loss: 0.0004593339116051495 - tot_loss: 1.0830891559895777 - acc: 0.6459466811751904 - val_acc: 0.6423481506616898
Repeatition 1 Epoch 265 / 500 
 - time: 1.820544719696045 - sq_loss: 0.0004577444992256314 - tot_loss: 1.0761407795244293 - acc: 0.64689880304679 - val_acc: 0.6423481506616898
Repeatition 1 Epoch 266 / 500 
 - time: 1.8336181640625 - sq_loss: 0.0004561738109978719 - tot_loss: 1.069

Repeatition 1 Epoch 307 / 500 
 - time: 1.8428137302398682 - sq_loss: 0.000403827698656056 - tot_loss: 0.829995940633915 - acc: 0.6829434167573449 - val_acc: 0.675262979300984
Repeatition 1 Epoch 308 / 500 
 - time: 1.8271145820617676 - sq_loss: 0.00040276780864561054 - tot_loss: 0.8250700412228782 - acc: 0.6833514689880305 - val_acc: 0.675941635561588
Repeatition 1 Epoch 309 / 500 
 - time: 1.8416175842285156 - sq_loss: 0.0004017155073419133 - tot_loss: 0.8201815133142883 - acc: 0.6838955386289445 - val_acc: 0.6766202918221921
Repeatition 1 Epoch 310 / 500 
 - time: 1.8357372283935547 - sq_loss: 0.0004006706768753524 - tot_loss: 0.815329823958752 - acc: 0.6844396082698585 - val_acc: 0.6769596199524941
Repeatition 1 Epoch 311 / 500 
 - time: 1.8083758354187012 - sq_loss: 0.0003996331565329117 - tot_loss: 0.8105147945732684 - acc: 0.6859357997823722 - val_acc: 0.6772989480827961
Repeatition 1 Epoch 312 / 500 
 - time: 1.8404457569122314 - sq_loss: 0.00039860288881951367 - tot_loss: 0.80

Repeatition 1 Epoch 353 / 500 
 - time: 1.8015720844268799 - sq_loss: 0.0003617231895843514 - tot_loss: 0.6372014851686879 - acc: 0.7130032644178455 - val_acc: 0.7020699015948422
Repeatition 1 Epoch 354 / 500 
 - time: 1.86785888671875 - sq_loss: 0.0003609389411208311 - tot_loss: 0.6336767752715659 - acc: 0.7140914036996736 - val_acc: 0.7024092297251442
Repeatition 1 Epoch 355 / 500 
 - time: 1.8494603633880615 - sq_loss: 0.00036015948529668877 - tot_loss: 0.6301765047967143 - acc: 0.7150435255712732 - val_acc: 0.7030878859857482
Repeatition 1 Epoch 356 / 500 
 - time: 1.8217995166778564 - sq_loss: 0.000359384778841982 - tot_loss: 0.626700475023524 - acc: 0.7158596300326442 - val_acc: 0.7034272141160502
Repeatition 1 Epoch 357 / 500 
 - time: 1.8372809886932373 - sq_loss: 0.00035861471608476087 - tot_loss: 0.6232484257056521 - acc: 0.7162676822633297 - val_acc: 0.7037665422463523
Repeatition 1 Epoch 358 / 500 
 - time: 1.8318641185760498 - sq_loss: 0.0003578492476118071 - tot_loss: 0.6

Repeatition 1 Epoch 399 / 500 
 - time: 1.8421916961669922 - sq_loss: 0.00032988283233662213 - tot_loss: 0.49747563876931045 - acc: 0.7399347116430903 - val_acc: 0.7268408551068883
Repeatition 1 Epoch 400 / 500 
 - time: 1.8302507400512695 - sq_loss: 0.0003292730677097921 - tot_loss: 0.494886107129626 - acc: 0.7399347116430903 - val_acc: 0.7275195113674924
Repeatition 1 Epoch 401 / 500 
 - time: 1.842233657836914 - sq_loss: 0.00032866622181768697 - tot_loss: 0.4923132297529748 - acc: 0.7407508161044614 - val_acc: 0.7281981676280964
Repeatition 1 Epoch 402 / 500 
 - time: 1.8108437061309814 - sq_loss: 0.00032806232173095124 - tot_loss: 0.48975687624969566 - acc: 0.7407508161044614 - val_acc: 0.7292161520190024
Repeatition 1 Epoch 403 / 500 
 - time: 1.8542132377624512 - sq_loss: 0.00032746133296627806 - tot_loss: 0.48721692216385787 - acc: 0.7410228509249184 - val_acc: 0.7288768238887003
Repeatition 1 Epoch 404 / 500 
 - time: 1.8430256843566895 - sq_loss: 0.0003268632545593561 - tot_lo